In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.dates as mdates

import tensorflow as tf
import seaborn as sns
from matplotlib import pyplot as plt
import os
from keras import mixed_precision

from datetime import datetime


'''

TODO: memory management!!! 

'''
from keras import backend as K

#os.chdir(".\\lib")
#print(os.getcwd())

#import models_base


import models_cuda
import ingest, predict


"""
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate 13GB of memory on the first GPU
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
"""


policy = mixed_precision.Policy('mixed_float16')

mixed_precision.set_global_policy(
    policy
)

## FLAGS ##
# RAS model output or USGS
USGS_FLAG = True

### HENRY RAS ###
csv = r"..\data\Henry_WSS_2017_2023.csv"
columns = {'Q': 'Discharge', 'WSS': 'Slope'}



## TESTS
1. WSS	        | Q
2. V	        | Q
3. WL	        | Q
4. WSS, V	    | Q
5. WSS, WL	    | Q
6. WSS, V, WL	| Q
6. WSS	        | WL
7. V	        | WL
8. Q	        | WL
9. WSS, V  	    | WL
10. WSS, Q	    | WL
11. WSS, V, Q	| WL


In [ ]:
WSS_Q = {"target": "Q", "features": { "WSS": "WSS"}}

WL_Q = {"target": "Q", "features": { "WL": "WL"}}

WSSWL_Q = {"target": "Q", "features": { "WSS": "WSS", "WL": "WL"}}

WSS_WL = {"target": "WL", "features": { "WSS": "WSS"}}

Q_WL = {"target": "WL", "features": { "Q": "Q"}}

WSSQ_WL = {"target": "WL", "features": { "WSS": "WSS", "Q": "Q"}}


tests= [WSS_Q,WL_Q,WSSWL_Q,WSS_WL,Q_WL,WSSQ_WL ]

In [ ]:

date = datetime.now().strftime("%B_%d_%Y_%H_%M")


target = "Discharge"
data_name = "Henry_RAS_2017_2023_" + f"cuda_testing"#"{date}"


train_range = ["1/1/2017 0:00","12/31/2021 23:45"]
test_range = ["1/1/2022 0:00", "12/31/2022 23:45"]
train_scaled, test_scaled, train_dates, test_dates, all_dates, scaler = ingest.ingest(csv, target, renames= columns, USGS_FLAG=USGS_FLAG, train_range= train_range, test_range= test_range)#train_test_ratio= 0.8)
trainX, trainY = ingest.reshape(train_scaled)#, timestep_type= "hr")
testX, testY = ingest.reshape(test_scaled)#, timestep_type= "hr")


model_names = ['Basic_LSTM', "GRU", 'Bidirectional_LSTM', 'Stacked_LSTM']


for model_name in model_names:
    model = models_cuda.prebuilt_models(model_name, trainX, trainY, epochs= 10, batch_size=32, loss= "nse", load_models=False, data= data_name)
    validation_loss = models_cuda.evaluate_model(model, testX, testY)
    models_cuda.plot_model(model_name, validation_loss, data_name)
    K.clear_session()

'''
tstart = '2022-03-18 00:00:00'
tend = '2022-04-07 00:00:00'
'''

tstart = "3/18/2022 0:00"
tend = "4/7/2022 23:45"
event_range = [tstart, tend]

for model_name in model_names:
    predicts = predict.predict(model_name, testX, data_name)
    predict.plot_predicts(model_name, predicts, testY, test_dates, data_name, event_range= event_range, event_plotstep= "Day")



In [ ]:

WSS_Q = {"target": "Q", "features": { "WSS": "WSS"}, "Name": "Q_WSS"}

WL_Q = {"target": "Q", "features": { "WL": "WL"}, "Name": "Q_WL"}

WSSWL_Q = {"target": "Q", "features": { "WSS": "WSS", "WL": "WL"}, "Name": "Q_WSS-WL"}

WSS_WL = {"target": "WL", "features": { "WSS": "WSS"}, "Name": "WL_WSS"}

Q_WL = {"target": "WL", "features": { "Q": "Q"}, "Name": "WL_Q"}

WSSQ_WL = {"target": "WL", "features": { "WSS": "WSS", "Q": "Q"}, "Name": "WL_WSS-Q"}


tests= [WSS_Q,WL_Q,WSSWL_Q,WSS_WL,Q_WL,WSSQ_WL ]



In [ ]:
for test in tests:
    dataname= "Henry_RAS_2017_2023_" + f"{test['Name']}"

    predicts = predict.predict(model_name, testX, data_name)
    
        
    plt.figure(figsize=(10, 8))
    sns.heatmap(feature_maps[0, :, :], annot=False, cmap='viridis')
    plt.ylabel('Timestep')
    plt.xlabel('Feature Map Index')
    plt.show()

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard


In [3]:
%tensorboard serve --logdir= "C:\Users\Mikey\Documents\Github\Hysterisis-ML-Modeling\lib\logs\Henry_RAS_2017_2023_Q_WSS" 

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] {serve,dev} ...
tensorboard: error: unrecognized arguments: C:\Users\Mikey\Documents\Github\Hysterisis-ML-Modeling\lib\logs\Henry_RAS_2017_2023_Q_WSS

In [4]:
%tensorboard --logdir=C:\Users\Mikey\Documents\Github\Hysterisis-ML-Modeling\lib\logs\Henry_RAS_2017_2023_Q_WSS  --helpfull --port= 6006

ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stdout:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

In [12]:
%tensorboard --version


ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stdout:
2.10.1